In [1]:
import altair as alt
import pandas as pd

# CUDA vs. OpenCL performance analysis example

First, generate parsed log dataset using

```bash
parse_fah_log dataframe --data-dir ../data --output ../data/benchmark_data PROJ17101 PROJ17102
```

In [2]:
data = pd.read_feather("../data/benchmark_data").set_index(["project", "run", "clone", "gen"]).sort_index()
data.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 102136 entries, ('PROJ17101', 0, 0, 0) to ('PROJ17102', 16, 918, 0)
Data columns (total 10 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   os                     102136 non-null  object 
 1   platform_name          102136 non-null  object 
 2   platform_vendor        102136 non-null  object 
 3   platform_version       102136 non-null  object 
 4   device_name            102136 non-null  object 
 5   device_vendor          102136 non-null  object 
 6   device_version         102136 non-null  object 
 7   device_driver_version  14813 non-null   object 
 8   cuda_enabled           102136 non-null  bool   
 9   perf_ns_per_day        100668 non-null  float64
dtypes: bool(1), float64(1), object(8)
memory usage: 7.8+ MB


In [3]:
device_name = "GeForce RTX 2080 Ti"

df = (data
      .pipe(lambda df: df[df["device_name"] == device_name])
      .assign(platform=lambda df: df["cuda_enabled"].replace({True: "CUDA", False: "OpenCL"}))
      .sample(5000)
     ).reset_index()

bars = alt.Chart().mark_bar().encode(
    x="platform:N",
    y=alt.Y("mean(perf_ns_per_day):Q", title="mean ns/day"),
    color='platform:N',
)

error_bars = alt.Chart().mark_errorbar(extent='ci').encode(
    x=alt.X("platform:N", title=None),
    y=alt.Y("perf_ns_per_day:Q", title=None)
)

alt.layer(bars, error_bars, data=df).facet(
    column="run:O",
    title=device_name
)

alt.FacetChart(...)

In [4]:
# List the 50 most "popular" cards (for BETA)

In [5]:
devices = data["device_name"].value_counts()
top_devices = devices[:50]
print(top_devices)

Tesla V100-SXM2-16GB      9729
GeForce RTX 2080 Ti       7606
A100-SXM4-40GB            6336
Hainan                    5078
GeForce GTX 1080 Ti       5063
GeForce GTX 1070          4721
Tesla K80                 3929
Ellesmere                 3691
GeForce GTX 1080          3666
Tesla M60                 3148
GeForce RTX 2070 SUPER    2899
GeForce RTX 2060 SUPER    2490
GeForce GTX 970           2370
GeForce RTX 2080          2237
GeForce RTX 2080 SUPER    2142
Capeverde                 2106
GeForce RTX 2060          2020
Tesla T4                  1990
GeForce GTX 1060 6GB      1987
GeForce RTX 2070          1885
gfx1010                   1597
GeForce GTX 1050 Ti       1201
Tesla V100-SXM2-32GB      1121
GeForce GTX 1660 Ti       1082
GeForce GTX 1070 Ti       1000
gfx900                     976
GeForce GTX 980            936
GeForce GTX 1660 SUPER     887
GeForce GTX 1060 3GB       870
GeForce GTX 980 Ti         797
GeForce GTX 960            777
gfx906                     651
Tesla V1

In [6]:
# Generate HTML files for each GPU

In [7]:
import os
from rich.progress import track

benchmark_dir = 'benchmark' # path to store benchmark output

devices = data["device_name"].value_counts()
for device_name in track(devices.keys(), description="Generating plots for all GPUs..."):
    # Create plot
    df = data[data["device_name"] == device_name].reset_index()

    bars = alt.Chart().mark_bar().encode(
        x="cuda_enabled:N",
        y=alt.Y("mean(perf_ns_per_day):Q", title="mean ns/day"),
        color='cuda_enabled:N',
    )

    error_bars = alt.Chart().mark_errorbar(extent='ci').encode(
        x=alt.X("cuda_enabled:N", title="cuda"),
        y=alt.Y("perf_ns_per_day:Q", title="")
    )

    chart = alt.layer(bars, error_bars, data=df).facet(
        column="run:O",
        title=device_name
    )

    html_filename = os.path.join(benchmark_dir, f'{device_name}.html')
    chart.save(html_filename)    

Output()

In [8]:
# Generate YAML and JSON files
import os
import yaml
import json
from rich.progress import track

benchmark_dir = 'benchmark' # path to store benchmark output

with open('run-metadata.yaml', 'rt') as infile:
    runs = yaml.load(infile.read())

print('Writing metadata...')
yaml_filename = os.path.join(benchmark_dir, f'run-metadata.yaml')
with open(yaml_filename, 'wt') as outfile:
    outfile.write(yaml.dump(runs))
    
json_filename = os.path.join(benchmark_dir, f'run-metadata.json')
with open(json_filename, 'wt') as outfile:
    outfile.write(json.dumps(runs))
    
    
osfullnames = {'linux' : 'linux2 4.19.76-linuxkit', 'win' : 'win32 10'}
devices = data["device_name"].value_counts()
for device_name in track(devices.keys(), description="Generating plots for all GPUs..."):
    device_df = data[data["device_name"] == device_name].reset_index()
    
    # Drop NaNs
    df.dropna(subset=['perf_ns_per_day'], inplace=True)
        
    # Create data record of summary statistics
    device_data = dict()
    device_data['device_name'] = device_name
    device_data['perf_ns_per_day'] = dict()        
    for run in runs.keys():
        run_index = int(run[3:])
        
        run_df = device_df[device_df["run"] == run_index].reset_index()
        device_data['perf_ns_per_day'][run] = { 
            'name' : runs[run]['name'],
            'num_atoms' : runs[run]['num_atoms'],
        }        
        
        for osname in osfullnames.keys():
            osfullname = osfullnames[osname]
            os_df = run_df[run_df["os"] == osfullname].reset_index()        

            perf = os_df['perf_ns_per_day']
                
            device_data['perf_ns_per_day'][run][osname] = {
                'mean' : float(perf.mean()),
                'std' : float(perf.std()),
                'min' : float(perf.min()),
                'max' : float(perf.max()),
                'nsamples' : len(perf),
            }        
            
    yaml_filename = os.path.join(benchmark_dir, f'{device_name}.yaml')
    with open(yaml_filename, 'wt') as outfile:
        outfile.write(yaml.dump(device_data))

    json_filename = os.path.join(benchmark_dir, f'{device_name}.json')
    with open(json_filename, 'wt') as outfile:
        outfile.write(json.dumps(device_data))



Writing metadata...


/Users/choderaj/miniconda/envs/perses/lib/python3.7/site-packages/ipykernel_launcher.py:10: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  # Remove the CWD from sys.path while we load stuff.


Output()

In [9]:
# Generate a simple index.html of all benchmark HTML files

In [10]:
contents = """

<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="utf-8">
<title>COVID Moonshot Sprint 4 Analysis</title>
<meta name="viewport" content="width=device-width, initial-scale=1.0, shrink-to-fit=no">
<!--[if lt IE 9]><script src="js/html5shiv-printshiv.js" media="all"></script><![endif]-->
<link href="https://fonts.googleapis.com/css2?family=Roboto&family=Roboto+Mono&display=swap" rel="stylesheet">
<style>
html * {
  font-family: 'Roboto', sans-serif;
}

span.updated {
    color: gray;
    font-style: italic;
}

table {
  border-collapse: collapse;
  width: 100%;
}

td, th {
  border: 1px solid #ddd;
  padding: 8px;
}

table.top20 td {
    font-size: 1.25em;
}

td.smiles {
    font-family: 'Roboto Mono', monospace;
}

td.binding span.estimate {
    white-space: nowrap;
}

td.binding span.point {
    font-weight: bold;
}

td.binding span.point > span.negative {
    color: green;
}

td.binding span.stderr {
    color: #555;
}

td.thumbnail {
    text-align: center;
}

td.thumbnail img {
    height: 100px;
}

tr:nth-child(even){background-color: #f2f2f2;}

tr:hover {background-color: #ddd;}

th {
  padding-top: 12px;
  padding-bottom: 12px;
  text-align: left;
  background-color: #1f77b4;
  color: white;
}

div.progress {
    border-style: solid;
    border-width: 2px;
    border-color: #1f77b4;
    background-color: white;
    padding: 3px;
}

div.progress > div.progress-bar {
    background-color: #ff7f0e;
    background-color: #1f77b4;
    color: white;
    font-size: 1.25em;
    font-weight: bold;
    font-style: italic;
    text-align: right;
    padding: 5px 20px 5px;
}
</style>
</head>
"""

from urllib.parse import quote
from datetime import datetime
contents += "<body>\n"
contents += "<h1>Folding@home benchmark suite (1710x): Preliminary analysis</h1>\n"
contents += f'<span class="updated">Last updated: {datetime.now()}</span>\n'
contents += f'<p>Metadata: <a href="run-metadata.yaml">[YAML]</a> <a href="run-metadata.yaml">[JSON]</a></p>\n'
contents += "<ul>\n"
for device_name in devices.keys():
    contents += f'  <li> <a href="{quote(device_name)}.html">[HTML]</a> <a href="{quote(device_name)}.yaml">[YAML]</a> <a href="{quote(device_name)}.json">[JSON]</a> : {device_name} </li>\n'
contents += "</ul>\n"
contents += "</body>\n"
with open(os.path.join(benchmark_dir, 'index.html'), 'wt') as outfile:
    outfile.write(contents)

# Transfer
```
aws s3 sync benchmark s3://fah-ws3/benchmark --profile jchodera --acl public-read
```    